In [1]:
import pandas as pd
import numpy as np
import ast

def AVERAGE(series):
    return np.mean(series.data.select_dtypes(include=[np.number]), axis=0)  # Compute mean of numeric data only

class Series:
    def __init__(self, series_header):
        self.data = df[series_header]

class SeriesRange:
    def __init__(self, series, start_index, end_index):
        # Concatenate the series and slice it to get the relevant data section
        combined_series = pd.concat([s.data for s in series], axis=1)
        self.data = combined_series.iloc[start_index:end_index + 1]  # Store the relevant data slice

# Securely parse and evaluate the AST
def eval_formula(formula):
    tree = ast.parse(formula, mode='eval')
    local_env = {
        'AVERAGE': AVERAGE,
        'SeriesRange': SeriesRange,
        'Series': Series,
        'np': np
    }
    compiled = compile(tree, filename="<ast>", mode='eval')
    result = eval(compiled, {'__builtins__': {}}, local_env)
    return result

# Example DataFrame
df = pd.DataFrame({
    'Period': pd.date_range(start='1/1/2020', periods=20),
    'Data_value': np.random.rand(20)
})

# Example formula
formula = "AVERAGE(SeriesRange(series=[Series(series_header='Period'), Series(series_header='Data_value')], start_index=0, end_index=10))"

result = eval_formula(formula)
result

Data_value    0.606741
dtype: float64

In [14]:
import pandas as pd
import numpy as np
import ast

import xlcalculator

# Create a dictionary to store the DataFrame
df_dict = {}
df_dict['Sheet1|col_1|1|1'] = pd.DataFrame({'col_1': [1.0, 3.0]})
df_dict['Sheet1|col_2|1|2'] = pd.DataFrame({'col_2': [2.0, 4.0]})

# Helper function to fetch and slice DataFrame
def fetch_df(identifier, index_range):
    df = df_dict[identifier]
    start, end = index_range
    return df.iloc[start:end+1]

# def AVERAGE(args):
#     identifiers, index_range = args
#     series_list = [fetch_df(identifier, index_range) for identifier in identifiers]
#     # Compute mean of numeric data only and return as scalar
#     return np.mean([np.mean(series.select_dtypes(include=[np.number])) for series in series_list]).item()

def AVERAGE(args):
    identifiers, index_range = args
    series_list = [fetch_df(identifier, index_range) for identifier in identifiers]
    numbers = [item for sublist in [series.select_dtypes(include=[np.number]).values.tolist() for series in series_list] for item in sublist]
    return xlcalculator.xlfunctions.statistics.AVERAGE(*numbers)

# Securely parse and evaluate the AST
def eval_formula(formula):
    tree = ast.parse(formula, mode='eval')
    local_env = {
        'AVERAGE': AVERAGE
    }
    compiled = compile(tree, filename="<ast>", mode='eval')
    result = eval(compiled, {'__builtins__': {}}, local_env)
    return result

# Example formula, second tuple represents the start index and end index of the pandas dataframe
formula = "AVERAGE((('Sheet1|col_1|1|1', 'Sheet1|col_2|1|2'), (0, 0)))"

# Evaluate the formula
result = eval_formula(formula)
assert result == 1.5  # This should pass if all is correct